# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import json

# Import API key
#from api_keys import g_key
#from vacation_config import api_key
#api_key
#api_key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
#Read data from the file which was created by the WeatherPy
#Location and name of the file Output/weather_city_data.csv
#Read file 
path_weather_file = "output_data/weather_city_data.csv"
weather_city_df = pd.read_csv(path_weather_file)
count_input_file =weather_city_df.count()
count_input_file

City           557
Lat            557
Lng            557
Max Temp       557
Humidity %     557
Cloudiness     557
Wind Speed     557
Country        551
Date           557
Temperature    557
dtype: int64

In [16]:
#input dataset:
weather_city_df.head()

,City,Lat,Lng,Max Temp,Humidity %,Cloudiness,Wind Speed,Country,Date,Temperature
0,Oranjemund,-28.5500,16.4333,56.25,37,0,9.33,NaN,1628998221,56.25
1,Nizwá,22.9333,57.5333,89.08,42,40,3.40,OM,1628998222,89.08
2,Saint-Philippe,-21.3585,55.7679,65.75,73,92,12.53,RE,1628998223,62.67
3,Baykit,61.6700,96.3700,58.39,86,100,4.72,RU,1628998225,58.39
4,Osuna,37.2377,-5.1075,87.82,24,37,2.80,ES,1628998226,79.48


In [4]:
weather_city_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity %', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Temperature'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Create a heat map that displays the humidity for every city from Part I.
#column filter = [[]]
Weather_Humidity = weather_city_df[["City","Lat","Lng","Humidity %"]]
Weather_Humidity

,City,Lat,Lng,Humidity %
0,Oranjemund,-28.5500,16.4333,37
1,Nizwá,22.9333,57.5333,42
2,Saint-Philippe,-21.3585,55.7679,73
3,Baykit,61.6700,96.3700,86
4,Osuna,37.2377,-5.1075,24
...,...,...,...,...
552,Key West,24.5557,-81.7826,79
553,North Bend,43.4065,-124.2243,94
554,Saiha,22.4833,92.9667,95
555,Strzelce Krajeńskie,52.8773,15.5298,89


In [18]:
locations= Weather_Humidity[["Lat","Lng"]]
locations

,Lat,Lng
0,-28.5500,16.4333
1,22.9333,57.5333
2,-21.3585,55.7679
3,61.6700,96.3700
4,37.2377,-5.1075
...,...,...
552,24.5557,-81.7826
553,43.4065,-124.2243
554,22.4833,92.9667
555,52.8773,15.5298


In [11]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [19]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations= Weather_Humidity[["Lat","Lng"]]
Humidity = Weather_Humidity["Humidity %"]
#gamp_lat_lng
#weight
# Create a  Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

#Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
narrow_weather_city_df = weather_city_df.query('Lat>0 and Lng>0 and Temperature>80')

#Drop any rows will null values.
narrow_weather_city_df.dropna(inplace=True)
narrow_weather_city_df.head(10)

<ipython-input-20-acc3ad5503ce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrow_weather_city_df.dropna(inplace=True)


,City,Lat,Lng,Max Temp,Humidity %,Cloudiness,Wind Speed,Country,Date,Temperature
1,Nizwá,22.9333,57.5333,89.08,42,40,3.40,OM,1628998222,89.08
77,George Town,5.4112,100.3354,83.89,77,40,3.00,MY,1628998239,82.60
80,Hong Kong,22.2855,114.1577,84.96,84,100,1.99,HK,1628998095,83.88
83,Riyadh,24.6877,46.7219,86.14,15,58,1.88,SA,1628998330,86.14
122,Butaritari,3.0707,172.7902,82.45,73,99,6.46,KI,1628998382,82.45
130,Pak Phanang,8.3511,100.2020,84.22,59,100,4.65,TH,1628998393,84.22
131,Dhidhdhoo,6.8833,73.1000,82.29,75,95,16.35,MV,1628998394,82.29
133,Kloulklubed,7.0419,134.2556,86.50,72,54,8.84,PW,1628998397,86.50
153,Haimen,23.1935,116.6142,89.49,71,56,11.05,CN,1628998423,89.49
160,Kumul,42.8000,93.4500,90.52,19,100,8.55,CN,1628998432,90.52


In [34]:

# Narrow weather city Graph

locations= narrow_weather_city_df[["Lat","Lng"]]
Humidity = narrow_weather_city_df["Humidity %"]
#gamp_lat_lng
#weight
# Create a  Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig
#narrow_weather_city_df
#Figure
#Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df= narrow_weather_city_df 
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity %,Cloudiness,Wind Speed,Country,Date,Temperature
1,Nizwá,22.9333,57.5333,89.08,42,40,3.40,OM,1628998222,89.08
77,George Town,5.4112,100.3354,83.89,77,40,3.00,MY,1628998239,82.60
80,Hong Kong,22.2855,114.1577,84.96,84,100,1.99,HK,1628998095,83.88
83,Riyadh,24.6877,46.7219,86.14,15,58,1.88,SA,1628998330,86.14
122,Butaritari,3.0707,172.7902,82.45,73,99,6.46,KI,1628998382,82.45


In [35]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

<ipython-input-35-480dc51f5844>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity %,Cloudiness,Wind Speed,Country,Date,Temperature,Hotel Name
1,Nizwá,22.9333,57.5333,89.08,42,40,3.40,OM,1628998222,89.08,
77,George Town,5.4112,100.3354,83.89,77,40,3.00,MY,1628998239,82.60,
80,Hong Kong,22.2855,114.1577,84.96,84,100,1.99,HK,1628998095,83.88,
83,Riyadh,24.6877,46.7219,86.14,15,58,1.88,SA,1628998330,86.14,
122,Butaritari,3.0707,172.7902,82.45,73,99,6.46,KI,1628998382,82.45,
130,Pak Phanang,8.3511,100.2020,84.22,59,100,4.65,TH,1628998393,84.22,
131,Dhidhdhoo,6.8833,73.1000,82.29,75,95,16.35,MV,1628998394,82.29,
133,Kloulklubed,7.0419,134.2556,86.50,72,54,8.84,PW,1628998397,86.50,
153,Haimen,23.1935,116.6142,89.49,71,56,11.05,CN,1628998423,89.49,
160,Kumul,42.8000,93.4500,90.52,19,100,8.55,CN,1628998432,90.52,


In [25]:
hotel_df.dropna(inplace=True)


<ipython-input-25-8cfd02ad83a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(inplace=True)


In [36]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
request_params_hotels = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
}
request_params_hotels

for index, row in hotel_df.iterrows():
    try:
            # get lat, lng from df
            lat = row["Lat"]
            lng = row["Lng"]

            # change location each iteration while leaving original params in place
            request_params_hotels["location"] = f"{lat},{lng}"

            # Use the search term: "International Airport" and our lat/lng
            #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

            # make request and print url
            hotel_name_data = requests.get(base_url, params=request_params_hotels).json()
    
            hotel_df.loc[index, "Hotel Name"] = hotel_name_data["results"][0]["name"]
    except IndexError:
            hotel_df.loc[index, "Hotel Name"] ="NaN"
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print( city_name.url)

    # convert to json
    #close_by_hotel_data = city_name.json()
    #name_address = name_address.json()


C:\Users\arnaz\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [38]:
# hotel name added to the dataset
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity %,Cloudiness,Wind Speed,Country,Date,Temperature,Hotel Name
1,Nizwá,22.9333,57.5333,89.08,42,40,3.40,OM,1628998222,89.08,Nizwa
77,George Town,5.4112,100.3354,83.89,77,40,3.00,MY,1628998239,82.60,George Town
80,Hong Kong,22.2855,114.1577,84.96,84,100,1.99,HK,1628998095,83.88,Hong Kong
83,Riyadh,24.6877,46.7219,86.14,15,58,1.88,SA,1628998330,86.14,Riyadh
122,Butaritari,3.0707,172.7902,82.45,73,99,6.46,KI,1628998382,82.45,Ukiangang
130,Pak Phanang,8.3511,100.2020,84.22,59,100,4.65,TH,1628998393,84.22,Pak Phanang
131,Dhidhdhoo,6.8833,73.1000,82.29,75,95,16.35,MV,1628998394,82.29,Dhidhdhoo
133,Kloulklubed,7.0419,134.2556,86.50,72,54,8.84,PW,1628998397,86.50,Klouklubed
153,Haimen,23.1935,116.6142,89.49,71,56,11.05,CN,1628998423,89.49,Shantou
160,Kumul,42.8000,93.4500,90.52,19,100,8.55,CN,1628998432,90.52,Hami


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map

#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
narrow_weather_city_df = weather_city_df.query('Lat>0 and Lng>0 and Temperature>80')

locations= narrow_weather_city_df[["Lat","Lng"]]
Humidity = narrow_weather_city_df["Humidity %"]
#gamp_lat_lng
#weight
# Create a  Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
                #[info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))